In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/MyDrive/Homework_1_ImageClassification/

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
#!unzip dataset.zip

In [ ]:
dataset_dir = 'training'

In [ ]:
# Plot example images from dataset
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

num_row = len(labels)//2
num_col = len(labels)//num_row
fig, axes = plt.subplots(num_row, num_col, figsize=(2*num_row,15*num_col))
for i in range(len(labels)):
  if i < len(labels):
    class_imgs = next(os.walk('{}/{}/'.format(dataset_dir, labels[i])))[2]
    class_img = class_imgs[0]
    img = Image.open('{}/{}/{}'.format(dataset_dir, labels[i], class_img))
    ax = axes[i//num_col, i%num_col]
    ax.imshow(np.array(img))
    ax.set_title('{}'.format(labels[i]))
plt.tight_layout()
plt.show()

Let the game begin

In [ ]:
import tensorflow as tf
tfk = tf.keras
print(tf.__version__)

In [ ]:
# Random seed for reproducibility
seed = 42

#random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

**Train Validation Splitting**

In [ ]:
image_generator_train = tfk.preprocessing.image.ImageDataGenerator(rescale = 1/255 , 
                                                             rotation_range = 30,
                                                             height_shift_range = 50,
                                                             width_shift_range= 50,
                                                             zoom_range = 0.3,
                                                             horizontal_flip = True,
                                                             vertical_flip=True,
                                                             fill_mode='reflect',
                                                             validation_split=0.1)

train_data = image_generator_train.flow_from_directory('training',
                                                 target_size=(256,256),
                                                 color_mode = 'rgb',
                                                 classes=None,
                                                 batch_size =64,
                                                 shuffle = True,
                                                 seed = seed,
                                                 subset = "training")

#For validation is necessary to create an ImageDataGenerator without data augmentation
image_generator_validation = tfk.preprocessing.image.ImageDataGenerator(rescale = 1/255 , 
                                                             validation_split=0.1)


validation_data = image_generator_validation.flow_from_directory('training',
                                                 target_size=(256,256),
                                                 color_mode = 'rgb',
                                                 classes=None,
                                                 batch_size =64,
                                                 shuffle = False,
                                                 seed = seed,
                                                 subset = "validation")


In [ ]:
print("Assigned labels")
print(train_data.class_indices)
print()
print("Target classes")
print(train_data.classes)

**CNN Model: build and fit**

In [ ]:
input_shape = (256,256,3)

In [ ]:
"""
This function creates the architecture of the our CNN
"""
def create_model(input_shape):
  model = tfk.Sequential()

  #Convolutional block for feature extraction

  #1
  model.add(tfk.layers.Conv2D(
      filters = 16,
      kernel_size = (3,3),
      strides = (1, 1),
      activation = "relu",
      padding = "same",
      kernel_initializer = tfk.initializers.GlorotUniform(seed),
      input_shape = input_shape
  ))
  model.add(tfk.layers.MaxPooling2D(pool_size=(2,2)))

  model.add(tfk.layers.BatchNormalization())

  #2
  model.add(tfk.layers.Conv2D(
      filters = 32,
      kernel_size = (3,3),
      strides = (1, 1),
      activation = "relu",
      padding = "same",
      kernel_initializer = tfk.initializers.GlorotUniform(seed)
  ))
  model.add(tfk.layers.MaxPooling2D(pool_size=(2,2)))

  model.add(tfk.layers.BatchNormalization())

  #3
  model.add(tfk.layers.Conv2D(
      filters = 64,
      kernel_size = (3,3),
      strides = (1, 1),
      activation = "relu",
      padding = "same",
      kernel_initializer = tfk.initializers.GlorotUniform(seed)
  ))
  model.add(tfk.layers.MaxPooling2D(pool_size=(2,2)))

  model.add(tfk.layers.BatchNormalization())

  #4
  model.add(tfk.layers.Conv2D(
      filters = 128,
      kernel_size = (3,3),
      activation = "relu",
      padding = "same",
      kernel_initializer = tfk.initializers.GlorotUniform(seed)
  ))
  model.add(tfk.layers.MaxPooling2D(pool_size=(2,2)))

  model.add(tfk.layers.BatchNormalization())

  #5
  model.add(tfk.layers.Conv2D(
      filters = 256,
      kernel_size = (3,3),
      activation = "relu",
      padding = "same",
      kernel_initializer = tfk.initializers.GlorotUniform(seed)
  ))
  model.add(tfk.layers.MaxPooling2D(pool_size=(2,2)))

  model.add(tfk.layers.BatchNormalization())

  model.add(tfk.layers.Flatten())

  #Fully connected block for classification

  model.add(tfk.layers.Dense(
      units = 512,
      activation = "relu",
      kernel_initializer=tfk.initializers.GlorotUniform(seed)
  ))

  model.add(tfk.layers.Dropout(0.5,seed=seed))


  #output layer
  model.add(tfk.layers.Dense(
      units = 14,
      activation = "softmax",
      kernel_initializer=tfk.initializers.GlorotUniform(seed)
  ))

  #compile model
  model.compile(loss=tfk.losses.CategoricalCrossentropy() ,
               optimizer = tfk.optimizers.Adam(),
               metrics='accuracy' )
  
  return model


In [ ]:
#callbacks (earlystopping)
callbacks = []
callbacks.append(tf.keras.callbacks.EarlyStopping(monitor ='val_loss', patience=5, restore_best_weights=True))

In [ ]:
#build the model
cnn_model = create_model(input_shape)
cnn_model.summary()

In [ ]:
#train the model
history = cnn_model.fit(x = train_data,
                        validation_data= validation_data,
                        epochs = 100,
                        callbacks = callbacks ).history

**Loss and Accuracy Plot to monitor overfitting**

In [ ]:
#Error function plot
import matplotlib.pyplot as plt
history_dict = history
history_dict
loss_values = history_dict['loss']
validation_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
plt.plot(epochs, validation_loss_values, label ='validation loss')
plt.xlabel('epochs')
plt.ylabel('error function ')
plt.title('Training and validation loss')

In [ ]:
#accuracy plot
accuracy_values = history_dict['accuracy']
validation_accuracy_values = history_dict['val_accuracy']

plt.clf()
plt.plot(epochs, accuracy_values, 'bo', label ='accurancy training')
plt.plot(epochs, validation_accuracy_values)
plt.xlabel('epochs')
plt.ylabel('accurancy')
plt.title('Training and validation accurancy')

In [ ]:
#save the model 
cnn_model.save("CNN_fourth_model")